[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AI-Hypercomputer/maxtext/blob/main/src/MaxText/examples/sft_qwen3_demo.ipynb)

# Qwen3-0.6B Supervised Fine-Tuning (SFT) Demo


In [3]:
#!git clone https://github.com/AI-Hypercomputer/maxtext.git
%cd /home/jimmytsai_google_com/workspace/maxtext

# Install uv, a fast Python package installer
#!pip install uv

# Install MaxText and its dependencies
#!uv pip install -e .[tpu] --resolution=lowest
#!python3 -m MaxText.install_maxtext_extra_deps

# Install vLLM for Jax and TPUs
#!uv pip install vllm-tpu==0.11.1
#!uv pip install --no-deps qwix==0.1.4
#!uv pip install -U ipywidgets

/mnt/disks/jimmy_workspace/maxtext


In [1]:
import os
import transformers

import MaxText
from MaxText import pyconfig
from MaxText.examples.sft_train_and_evaluate import evaluate_model, get_test_dataset
from MaxText.integration.tunix.tunix_adapter import TunixMaxTextAdapter
from MaxText.sft import sft_trainer

from tunix.rl.rollout.vllm_rollout import VllmRollout

from datetime import datetime
from flax import nnx
from huggingface_hub import login

MAXTEXT_REPO_ROOT = os.path.dirname(MaxText.__file__)
print(f"MaxText installation path: {MAXTEXT_REPO_ROOT}")
import dotenv
dotenv.load_dotenv(override=True)
HF_TOKEN = os.environ.get("HF_TOKEN")
if HF_TOKEN:
  login(token=HF_TOKEN)
  print("Authenticated with Hugging Face successfully!")
else:
  print("Authentication failed: HF_TOKEN is not set.")

2025-12-22 08:53:16.858283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766393596.871686 2957460 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766393596.876154 2957460 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766393596.887912 2957460 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766393596.887927 2957460 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766393596.887929 2957460 computation_placer.cc:177] computation placer alr

MaxText installation path: /mnt/disks/jimmy_workspace/maxtext/src/MaxText
Authenticated with Hugging Face successfully!


In [2]:
MODEL_NAME = "qwen3-4b"
TOKENIZER_PATH = "Qwen/Qwen3-4B"
tokenizer = transformers.AutoTokenizer.from_pretrained(
    TOKENIZER_PATH,
    token=HF_TOKEN,
)

# This colab will download the checkpoint from HF and store at `MODEL_CHECKPOINT_PATH`
MODEL_CHECKPOINT_PATH = f"{MAXTEXT_REPO_ROOT}/qwen_checkpoint-4b"

RUN_NAME = datetime.now().strftime("%Y-%m-%d-%H-%m-%S")

# This is the directory where the fine-tuned model checkpoint will be saved
BASE_OUTPUT_DIRECTORY = f"/mnt/disks/jimmy_workspace/maxtext_qwen-4b"
print(f"Model checkpoint can be found at: {MODEL_CHECKPOINT_PATH}/0/items")

Model checkpoint can be found at: /mnt/disks/jimmy_workspace/maxtext/src/MaxText/qwen_checkpoint-4b/0/items


In [ ]:
# !python3 -m MaxText.utils.ckpt_conversion.to_maxtext \
#     $MAXTEXT_REPO_ROOT/configs/base.yml \
#     model_name=$MODEL_NAME \
#     base_output_directory=$MODEL_CHECKPOINT_PATH \
#     hf_access_token=$HF_TOKEN \
#     use_multimodal=false \
#     scan_layers=true

In [10]:
#DATASET_NAME = "openai/gsm8k"
DATASET_NAME = "/home/jimmytsai_google_com/workspace/tencent_sft/clid-data"
TRAIN_DATA_SPLIT = "train"
TEST_DATA_SPLIT = "test"
HF_DATA_DIR = "data"
TRAIN_DATA_COLUMNS = ["messages"]
#CHAT_TEMPLATE_PATH = f"{MAXTEXT_REPO_ROOT}/examples/chat_templates/math_qa.json"
NUM_TEST_SAMPLES = 20  # Total number of samples to test
BATCH_SIZE = 1  # Number of test samples to process in a batch
config = pyconfig.initialize(
    [
        "",
        f"{MAXTEXT_REPO_ROOT}/configs/sft.yml",
        f"load_parameters_path={MODEL_CHECKPOINT_PATH}/0/items",
        f"model_name={MODEL_NAME}",
        f"hf_access_token={HF_TOKEN}",
        f"base_output_directory={BASE_OUTPUT_DIRECTORY}",
        f"run_name={RUN_NAME}",
        f"tokenizer_path={TOKENIZER_PATH}",
        f"hf_path={DATASET_NAME}",
        f"train_split={TRAIN_DATA_SPLIT}",
        f"hf_data_dir={HF_DATA_DIR}",
        f"train_data_columns={TRAIN_DATA_COLUMNS}",
        "steps=400",
        "per_device_batch_size=1",
        "max_target_length=1024",
        "learning_rate=3e-6",
        "weight_dtype=bfloat16",
        "dtype=bfloat16",
        #f"chat_template_path={CHAT_TEMPLATE_PATH}",
    ]
)

Jax distributed system is already initialized.


INFO:MaxText.pyconfig:Config param act_quantization_calibration_method: absmax
INFO:MaxText.pyconfig:Config param activations_in_float32: False
INFO:MaxText.pyconfig:Config param adam_b1: 0.9
INFO:MaxText.pyconfig:Config param adam_b2: 0.95
INFO:MaxText.pyconfig:Config param adam_eps: 1e-08
INFO:MaxText.pyconfig:Config param adam_eps_root: 0.0
INFO:MaxText.pyconfig:Config param adam_weight_decay: 0.1
INFO:MaxText.pyconfig:Config param add_bos: True
INFO:MaxText.pyconfig:Config param add_eos: True
INFO:MaxText.pyconfig:Config param allow_split_physical_axes: False
INFO:MaxText.pyconfig:Config param ar_cache_axis_order: 1,2,0,3
INFO:MaxText.pyconfig:Config param async_checkpointing: True
INFO:MaxText.pyconfig:Config param attention: autoselected
INFO:MaxText.pyconfig:Config param attention_bias: False
INFO:MaxText.pyconfig:Config param attention_sink: False
INFO:MaxText.pyconfig:Config param attention_type: global
INFO:MaxText.pyconfig:Config param attn_logits_soft_cap: None
INFO:MaxText

In [20]:
test_dataset = get_test_dataset(config, tokenizer)
test_dataset = test_dataset[:NUM_TEST_SAMPLES]
test_dataset = test_dataset.to_iter_dataset().batch(BATCH_SIZE, drop_remainder=True)
TOTAL_BATCHES = NUM_TEST_SAMPLES // BATCH_SIZE
print(
    f"Processing {NUM_TEST_SAMPLES} examples with a batch size of {BATCH_SIZE}. This will result in {TOTAL_BATCHES} total batches for the test run."
)

Processing 20 examples with a batch size of 1. This will result in 20 total batches for the test run.


In [11]:
trainer, mesh = sft_trainer.setup_trainer_state(config)
tunix_model = TunixMaxTextAdapter(trainer.model)
vllm_rollout = VllmRollout(
    model=tunix_model,
    tokenizer=tokenizer,
    cache_config_or_size=1280,
    mesh=mesh,
    model_version=TOKENIZER_PATH,
    hbm_utilization=0.8,
    init_with_random_weights=True,
    tpu_backend_type="jax",
)

Num_devices: 8, shape (1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Num_devices: 8, shape (1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1)


ERROR:absl:PackAndBatchOperation is deprecated. Please use lazy_dataset.FirstFitPackIterDataset instead.
`torch_dtype` is deprecated! Use `dtype` instead!


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0

In [ ]:
print("Running Pre-SFT Evaluation...")
score = evaluate_model(test_dataset, vllm_rollout, debug=True)

In [ ]:
print("========================= Score for PRE-SFT Evaluation =========================")
print(f"Percentage of test samples where the model produced the correct numerical answer: {score['correct']}%")
print(
    f"Percentage of test samples where the model produced the numerical answer within 10%: {score['partially_correct']}%"
)
print(
    f"Percentage of test samples where the model's output adheres to the expected structure: {score['correct_format']}%"
)

## SFT Training

In [12]:
print("Starting SFT Training...")
trainer = sft_trainer.train_model(config, trainer, mesh)
print("SFT Training Complete!")

Starting SFT Training...


Training:   0%|          | 0/400 [00:00<?, ?step/s]

Per train step:
 Total TFLOPs: 25.64 
 split as 96.38% learnable weight flops and 3.62% attention flops
number parameters: 4.022 billion


2025-12-22 02:36:52.651543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766371012.664715 2832566 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766371012.668903 2832566 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766371012.680155 2832566 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766371012.680168 2832566 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766371012.680170 2832566 computation_placer.cc:177] computation placer alr


Memstats: After params initialized:
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_0(process=0,(0,0,0,0))
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_1(process=0,(1,0,0,0))
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_2(process=0,(0,1,0,0))
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_3(process=0,(1,1,0,0))
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_4(process=0,(0,2,0,0))
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_5(process=0,(1,2,0,0))
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_6(process=0,(0,3,0,0))
	Using (GB) 25.61 / 31.25 (81.952000%) on TPU_7(process=0,(1,3,0,0))
completed step: 1, seconds: 16.095, TFLOP/s/device: 1.593, Tokens/s/device: 63.624, total_weights: 1113, loss: 6.389
completed step: 2, seconds: 2.514, TFLOP/s/device: 10.198, Tokens/s/device: 407.288, total_weights: 1035, loss: 6.739
completed step: 3, seconds: 0.450, TFLOP/s/device: 56.999, Tokens/s/device: 2276.374, total_weights: 1161, loss: 6.159
completed step: 4, seconds: 0.248, TFLOP/s/device: 103.20

SFT Training Complete!


## Evaluation after SFT Training

In [ ]:
print("Running Post-SFT Evaluation...")
model = TunixMaxTextAdapter(trainer.model)
state = nnx.state(model)
#vllm_rollout.update_params(state)
#score = evaluate_model(test_dataset, vllm_rollout, debug=True)
vllm_rollout = VllmRollout(
    model=tunix_model,
    tokenizer=tokenizer,
    cache_config_or_size=1280,
    mesh=mesh,
    model_version=TOKENIZER_PATH,
    hbm_utilization=0.8,
    init_with_random_weights=True,
    tpu_backend_type="jax",
)
vllm_rollout.update_params(state)

In [ ]:
print("========================= Score for POST-SFT Evaluation =========================")
print(f"Percentage of test samples where the model produced the correct numerical answer: {score['correct']}%")
print(
    f"Percentage of test samples where the model produced the numerical answer within 10%: {score['partially_correct']}%"
)
print(
    f"Percentage of test samples where the model's output adheres to the expected structure: {score['correct_format']}%"
)

In [ ]:
nnx.display(state)

: 